- Put your password for sudo in `./sudo_password.txt`, since it will be needed for running CICFlowMeter.
- PCAP from original CICIDS2017 must be reordered by `reordercap`

In [15]:
PCAP_INPUT = "/mnt/extra/datasets/pcap/downloading/Friday-WorkingHours-Reordered.pcap"
FLOW_INPUT = "/mnt/extra/datasets/fixed-2/friday.csv"
OUTPUT_FOLDER = "/mnt/extra/datasets/processed"
BASE_OUTPUT_NAME = "friday"
EXTRACT_FLOWS_OUTPUT_FOLDER = OUTPUT_FOLDER

VALIDATION_PERCENTAGE = 0.1
TEST_PERCENTAGE = 0.1

GRACE_PERCENTAGE = 0.02
VALIDATION_PERCENTAGE += GRACE_PERCENTAGE
TEST_PERCENTAGE += GRACE_PERCENTAGE

LABEL_MAPPING = {
    'BENIGN': 'Not Portscan',
    'Portscan': 'Portscan',
    'DDoS': 'Not Portscan',
    'Botnet - Attempted': 'Not Portscan',
    'Botnet': 'Not Portscan',
}

In [2]:
import os

def extract_flows_from_pcap(filepath: str, output_name: str):
    basename = os.path.basename(filepath)
    flow_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_flow.csv"
    index_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_index.txt"

    os.system(f'sudo -S java -Djava.library.path="./Tools/jnetpcap-1.4.r1500" -jar ./Tools/CICFlowMeterV3-0.0.4-SNAPSHOT.jar {filepath} {EXTRACT_FLOWS_OUTPUT_FOLDER} < ./sudo_password.txt')
    
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv", flow_filename)
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv_index.txt", index_filename)

    return flow_filename, index_filename

# Pre-Process Dataset
- Adjust labels to make it compatible with research scope
- Remove rows that makes mapping between packets and flows hard

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
distinguishing_keys = [
    'Src IP',
    'Src Port',
    'Dst IP',
    'Dst Port',
    'Timestamp',
]

In [5]:
compiled = pd.read_csv(FLOW_INPUT, parse_dates=['Timestamp'])
compiled

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448,1152,403,0,201.500000,204.724205,72,72,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772,3,112740690,3.636796e+06,6.848761e+06,16399772,3,112740348,7.516023e+06,8.323385e+06,16400110,3,16,16,0,0,0,0,1024,512,0.283837,0.141919,0,403,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0,0,0,0,0,0,0,134,0,24,377,2079,15,32,3.594286e+02,1.199802e+01,380,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448,5056,403,0,201.500000,204.724205,316,316,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782,2,112740560,3.636792e+06,6.848777e+06,16399782,2,112740242,7.516016e+06,8.323376e+06,16400110,4,16,16,0,0,0,0,1024,512,0.283838,0.141919,0,403,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0,0,0,0,0,0,0,134,0,105,955,2079,15,32,3.202857e+02,1.574499e+01,330,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,BENIGN,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,4.790898,2.091128e+05,1.395543e+06,20757030,0,113757377,2.091128e+05,1.395543e+06,20757030,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,4.790898,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9.361829e+06,7.324646e+06,18851791,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,BENIGN,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151,0,227,37,95.750000,55.785320,0,0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791,0,91997219,2.377189e+05,1.511622e+06,19776791,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37,227,95.750000,55.785320,3112.001938,0,0,0,0,0,0,0,0,0.000000,95.750000,95.750000,0.000000,1857,19,9453,0,0,0,1,95,0,0,0,0,387,8,9.801664e+06,1.152782e+07,24721964,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,BENIGN,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,6,288,288,48,48,48.000000,0.000000,48,48,48.000000,0.000000,8.601371e+00,0.179195,6.087825e+06,1.953823e+07,64974431,1

In [8]:
compiled['Label'].value_counts(dropna=False)

Label
BENIGN                288544
Portscan              159066
DDoS                   95144
Botnet - Attempted      4067
Botnet                   736
Name: count, dtype: int64

In [9]:
compiled['Label'] = compiled['Label'].map(LABEL_MAPPING)
compiled = compiled.dropna(subset=['Label'])
compiled['Label'].value_counts(dropna=False)

Label
Not Portscan    388491
Portscan        159066
Name: count, dtype: int64

In [10]:
compiled_keys = compiled[distinguishing_keys]
has_duplicate = (compiled_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

compiled_unique = compiled[~compiled_keys.duplicated(keep=False)]
compiled_unique

Has Duplicates? False


,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448,1152,403,0,201.500000,204.724205,72,72,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772,3,112740690,3.636796e+06,6.848761e+06,16399772,3,112740348,7.516023e+06,8.323385e+06,16400110,3,16,16,0,0,0,0,1024,512,0.283837,0.141919,0,403,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0,0,0,0,0,0,0,134,0,24,377,2079,15,32,3.594286e+02,1.199802e+01,380,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,Not Portscan,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448,5056,403,0,201.500000,204.724205,316,316,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782,2,112740560,3.636792e+06,6.848777e+06,16399782,2,112740242,7.516016e+06,8.323376e+06,16400110,4,16,16,0,0,0,0,1024,512,0.283838,0.141919,0,403,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0,0,0,0,0,0,0,134,0,105,955,2079,15,32,3.202857e+02,1.574499e+01,330,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,Not Portscan,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,4.790898,2.091128e+05,1.395543e+06,20757030,0,113757377,2.091128e+05,1.395543e+06,20757030,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,4.790898,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9.361829e+06,7.324646e+06,18851791,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,Not Portscan,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151,0,227,37,95.750000,55.785320,0,0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791,0,91997219,2.377189e+05,1.511622e+06,19776791,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37,227,95.750000,55.785320,3112.001938,0,0,0,0,0,0,0,0,0.000000,95.750000,95.750000,0.000000,1857,19,9453,0,0,0,1,95,0,0,0,0,387,8,9.801664e+06,1.152782e+07,24721964,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,Not Portscan,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,6,288,288,48,48,48.000000,0.000000,48,48,48.000000,0.000000,8.601371e+00,0.179195,6.087825e+06

In [11]:
compiled_unique.to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_preprocess_1.csv", index=False)

In [12]:
compiled = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_preprocess_1.csv", parse_dates=['Timestamp'])
compiled

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448,1152,403,0,201.500000,204.724205,72,72,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772,3,112740690,3.636796e+06,6.848761e+06,16399772,3,112740348,7.516023e+06,8.323385e+06,16400110,3,16,16,0,0,0,0,1024,512,0.283837,0.141919,0,403,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0,0,0,0,0,0,0,134,0,24,377,2079,15,32,3.594286e+02,1.199802e+01,380,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,Not Portscan,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448,5056,403,0,201.500000,204.724205,316,316,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782,2,112740560,3.636792e+06,6.848777e+06,16399782,2,112740242,7.516016e+06,8.323376e+06,16400110,4,16,16,0,0,0,0,1024,512,0.283838,0.141919,0,403,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0,0,0,0,0,0,0,134,0,105,955,2079,15,32,3.202857e+02,1.574499e+01,330,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,Not Portscan,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,4.790898,2.091128e+05,1.395543e+06,20757030,0,113757377,2.091128e+05,1.395543e+06,20757030,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,4.790898,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9.361829e+06,7.324646e+06,18851791,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,Not Portscan,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151,0,227,37,95.750000,55.785320,0,0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791,0,91997219,2.377189e+05,1.511622e+06,19776791,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37,227,95.750000,55.785320,3112.001938,0,0,0,0,0,0,0,0,0.000000,95.750000,95.750000,0.000000,1857,19,9453,0,0,0,1,95,0,0,0,0,387,8,9.801664e+06,1.152782e+07,24721964,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,Not Portscan,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,6,288,288,48,48,48.000000,0.000000,48,48,48.000000,0.000000,8.601371e+00,0.179195,6.087825e+06

In [13]:
compiled['Label'].value_counts(dropna=False)

Label
Not Portscan    388491
Portscan        159066
Name: count, dtype: int64

# Run CICFlowMeter on PCAP Dataset
- Get features from current version of CICFlowMeter (e.g. to fix bugs in features caused by old CICFlowMeter)

In [16]:
rerun1_flow_file, rerun1_index_file = extract_flows_from_pcap(PCAP_INPUT, f"{BASE_OUTPUT_NAME}_rerun_1")

[sudo] password for meervix: 

cic.cs.unb.ca.ifm.Cmd You select: /mnt/extra/datasets/pcap/downloading/Friday-WorkingHours-Reordered.pcap
cic.cs.unb.ca.ifm.Cmd Out folder: /mnt/extra/datasets/processed
cic.cs.unb.ca.ifm.Cmd CICFlowMeter received 1 pcap file
Working on... Friday-WorkingHours-Reordered.pcap
Friday-WorkingHours-Reordered.pcap is done. total 548603 flows 
Packet stats: Total=9997874,Valid=9938308,Discarded=59566
-------------------------------------------------------------------------------


In [17]:
rerun1_flow_file = f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_rerun_1_flow.csv"
rerun1_index_file = f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_rerun_1_index.txt"

In [18]:
flowmeter = pd.read_csv(rerun1_flow_file, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,NeedManualLabel
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,NeedManualLabel
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,NeedManualLabel
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791.0,0.0,91997219.0,2.377189e+05,1.511622e+06,19776791.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37.0,227.0,95.750000,55.785320,3112.001938,

In [19]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter = flowmeter[~flowmeter_keys.duplicated(keep=False)]
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,NeedManualLabel
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,NeedManualLabel
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,NeedManualLabel
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791.0,0.0,91997219.0,2.377189e+05,1.511622e+06,19776791.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37.0,227.0,95.750000,55.785320,3112.001938,

# Annotate Rerunned CSV using Dataset's CSV

In [20]:
compiled_keys = compiled[distinguishing_keys + ['Label']]
compiled[compiled_keys.duplicated(keep=False)]

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category


In [21]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter[flowmeter_keys.duplicated(keep=False)]

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label


In [22]:
left_merge = flowmeter_keys.reset_index().merge(compiled_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.50,56108,192.168.10.3,3268,2017-07-07 11:59:50.315195,Not Portscan,both
1,192.168.10.50,42144,192.168.10.3,389,2017-07-07 11:59:50.316273,Not Portscan,both
2,8.6.0.1,0,8.0.6.4,0,2017-07-07 12:00:31.388567,Not Portscan,both
3,192.168.10.25,5353,224.0.0.251,5353,2017-07-07 12:00:42.903850,Not Portscan,both
4,192.168.10.25,123,17.253.14.125,123,2017-07-07 12:00:42.430758,Not Portscan,both
...,...,...,...,...,...,...,...
548597,172.16.0.1,64318,192.168.10.50,2222,2017-07-07 18:09:11.499555,Portscan,both
548598,172.16.0.1,33248,192.168.10.50,9040,2017-07-07 17:54:38.857717,Portscan,both
548599,172.16.0.1,44033,192.168.10.50,1272,2017-07-07 17:52:03.285040,Portscan,both


In [23]:
flowmeter_ignored = left_merge[left_merge['_merge'] == 'left_only']
flowmeter_ignored_idx = flowmeter_ignored.index
flowmeter_merged = flowmeter.drop(index=flowmeter_ignored_idx)

In [24]:
flowmeter_merged['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']

In [25]:
flowmeter_merged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,Not Portscan
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,Not Portscan
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,Not Portscan
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791.0,0.0,91997219.0,2.377189e+05,1.511622e+06,19776791.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37.0,227.0,95.750000,55.785320,3112.001938,0,0,0,0,0

In [26]:
flowmeter_merged['Label'].value_counts()

Label
Not Portscan    387727
Portscan        159066
Name: count, dtype: int64

In [27]:
flowmeter_merged.reset_index().to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated.csv", index=False)

## Load Checkpoint

In [28]:
flowmeter = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated.csv", index_col=0, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,Not Portscan
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,Not Portscan
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,Not Portscan
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791.0,0.0,91997219.0,2.377189e+05,1.511622e+06,19776791.0,0.0,0.0,0.000000e+00,0.000000e+

# Train-Test Split
## Test Split

In [29]:
flowmeter['Timestamp End'] = flowmeter['Timestamp'] + pd.to_timedelta(flowmeter['Flow Duration'], unit="us")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,Not Portscan,2017-07-07 12:01:43.055885
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,Not Portscan,2017-07-07 12:01:43.056833
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,Not Portscan,2017-07-07 12:02:25.145944
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+0

In [30]:
tagged = flowmeter[flowmeter['Label'] == 'Portscan']
tagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69666,172.16.0.1-192.168.10.50-33830-4848-6,172.16.0.1,33830,192.168.10.50,4848,6,2017-07-07 17:52:58.173454,44,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45454.545455,44.0,0.0,44.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,24,20,22727.272727,22727.272727,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,0,0,24,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,44,Portscan,2017-07-07 17:52:58.173498
69667,172.16.0.1-192.168.10.50-33830-1272-6,172.16.0.1,33830,192.168.10.50,1272,6,2017-07-07 17:52:58.178547,43,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46511.627907,43.0,0.0,43.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,24,20,23255.813953,23255.813953,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,0,0,24,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,43,Portscan,2017-07-07 17:52:58.178590
69668,172.16.0.1-192.168.10.50-39492-5862-6,172.16.0.1,39492,192.168.10.50,5862,6,2017-07-07 17:52:33.479484,54,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37037.037037,54.0,0.0,54.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,24,20,18518.518519,18518.518519,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,0,0,24,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,54,Portscan,2017-07-07 17:52:33.479538
69669,172.16.0.1-192.168.10.50-33830-2605-6,172.16.0.1,33830,192.168.10.50,2605,6,2017-07-07 17:52:58.179078,33,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60606.060606,33.0,0.0,33.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,24,20,30303.030303,30303.030303,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,0,0,24,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,33,Portscan,2017-07-07 17:52:58.179111
69671,172.16.0.1-192.168.10.50-39070-17988-6,172.16.0.1,39070,192.168.10.50,17988,6,2017-07-07 17:52:11.570780,50,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40000.000000,50.0,0.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,24,20,20000.000000,20000.000000,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,0,0,24,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,50,Portscan,2017-07-07 17:52:11.570830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [31]:
untagged = flowmeter[flowmeter['Label'] != 'Portscan']
untagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,3.145010e+04,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,Not Portscan,2017-07-07 12:01:43.055885
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,3.061946e+04,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,Not Portscan,2017-07-07 12:01:43.056833
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,Not Portscan,2017-07-07 12:02:25.145944
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.51162

In [32]:
import numpy as np

min_time = np.min(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Whole flows are from \"{min_time}\" to \"{max_time}\"")

Whole flows are from "2017-07-07 08:59:50.315195" to "2017-07-07 17:02:40.977798"


In [34]:
min_time = np.min(tagged['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(tagged['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-07 13:55:32.117090" to "2017-07-07 15:23:53.256235"


In [36]:
firewall_on_min_time = pd.Timestamp("2017-07-07 13:55:00") + pd.Timedelta(hours=3)
firewall_on_max_time = pd.Timestamp("2017-07-07 14:36:00") + pd.Timedelta(hours=3)
firewall_off_min_time = pd.Timestamp("2017-07-07 14:51:00") + pd.Timedelta(hours=3)
firewall_off_max_time = pd.Timestamp("2017-07-07 15:30:00") + pd.Timedelta(hours=3)

In [ ]:
firewall_on = tagged[(firewall_on_min_time <= tagged['Timestamp']) & (tagged['Timestamp End'] <= firewall_on_max_time)]
firewall_on_untagged = untagged[(firewall_on_min_time <= untagged['Timestamp']) & (untagged['Timestamp End'] <= firewall_on_max_time)]

In [38]:
firewall_off = tagged[(firewall_off_min_time <= tagged['Timestamp']) & (tagged['Timestamp End'] <= firewall_off_max_time)]
firewall_off_untagged = untagged[(firewall_off_min_time <= untagged['Timestamp']) & (untagged['Timestamp End'] <= firewall_off_max_time)]

In [39]:
len(firewall_on) + len(firewall_off) == tagged['Label'].value_counts(dropna=False)['Portscan']

True

In [40]:
benign_test = int(len(untagged) * TEST_PERCENTAGE)
firewall_on_test = int(len(firewall_on) * TEST_PERCENTAGE)
firewall_off_test = int(len(firewall_off) * TEST_PERCENTAGE)

print("Test Dataset Numbers:")
print(f"Benign: {benign_test}")
print(f"Firewall On: {firewall_on_test}")
print(f"Firewall Off: {firewall_off_test}")

Test Dataset Numbers:
Benign: 46527
Firewall On: 41
Firewall Off: 19046


In [ ]:
# Firewall On
firewall_on_timestamps = firewall_on.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(firewall_on_timestamps)):
    if firewall_on_timestamps[i-1] > firewall_on_timestamps[i]:
        print("NOT SORTED")

cnt = -1
firewall_on_test_df = None
firewall_on_chosen_time = None
for tmp in firewall_on_timestamps[::-1]:
    if cnt >= firewall_on_test:
        break
    firewall_on_test_df = firewall_on[(tmp <= firewall_on['Timestamp'])]
    firewall_on_chosen_time = tmp
    cnt = len(firewall_on_test_df)

firewall_on_untagged_test_df = firewall_on_untagged[(firewall_on_chosen_time <= firewall_on_untagged['Timestamp'])]

print(firewall_on_test)
print(firewall_on_chosen_time)
print(len(firewall_on_test_df))
print(len(firewall_on_untagged_test_df))

41
2017-07-07 17:15:21.047424
41
13530


In [57]:
# Firewall Off
firewall_off_timestamps = firewall_off.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(firewall_off_timestamps)):
    if firewall_off_timestamps[i-1] > firewall_off_timestamps[i]:
        print("NOT SORTED")

tmp = firewall_off_timestamps[::-1][18946]
firewall_off_test_df = firewall_off[(tmp <= firewall_off['Timestamp'])]
firewall_off_untagged_test_df = firewall_off_untagged[(tmp <= firewall_off_untagged['Timestamp'])]
print(tmp)
print(f"{len(firewall_off_test_df)}/{firewall_off_test}")

2017-07-07 18:08:16.544542
19046/19046


In [58]:
benign_left = benign_test - (len(firewall_on_untagged_test_df) + len(firewall_off_untagged_test_df))
benign_left

14676

In [59]:
benign_timestamps = untagged.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(benign_timestamps)):
    if benign_timestamps[i-1] > benign_timestamps[i]:
        print("NOT SORTED")

In [84]:
tmp = benign_timestamps[::-1][14669]
benign_test_df = untagged[(tmp <= untagged['Timestamp'])]
print(tmp)
print(f"{len(benign_test_df)}/{benign_left}")

2017-07-07 19:29:20.650839
14676/14676


In [85]:
firewall_on_test_df['Label'].value_counts()

Label
Portscan    41
Name: count, dtype: int64

In [86]:
firewall_on_untagged_test_df['Label'].value_counts()

Label
Not Portscan    13530
Name: count, dtype: int64

In [87]:
firewall_off_test_df['Label'].value_counts()

Label
Portscan    19046
Name: count, dtype: int64

In [88]:
firewall_off_untagged_test_df['Label'].value_counts()

Label
Not Portscan    18321
Name: count, dtype: int64

In [89]:
benign_test_df['Label'].value_counts()

Label
Not Portscan    14676
Name: count, dtype: int64

In [96]:
df_test = pd.concat([
    firewall_on_test_df,
    firewall_on_untagged_test_df,
    firewall_off_test_df,
    firewall_off_untagged_test_df,
    benign_test_df,
])
df_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93277,172.16.0.1-192.168.10.50-58377-80-6,172.16.0.1,58377,192.168.10.50,80,6,2017-07-07 17:15:45.540329,834,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,3597.122302,4.170000e+02,4.101219e+02,707.0,127.0,834.0,8.340000e+02,0.000000e+00,834.0,834.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2398.081535,1199.040767,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,834,Portscan,2017-07-07 17:15:45.541163
98282,172.16.0.1-192.168.10.50-62805-80-6,172.16.0.1,62805,192.168.10.50,80,6,2017-07-07 17:15:57.176074,674,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4451.038576,3.370000e+02,3.337544e+02,573.0,101.0,674.0,6.740000e+02,0.000000e+00,674.0,674.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2967.359050,1483.679525,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,674,Portscan,2017-07-07 17:15:57.176748
120211,172.16.0.1-192.168.10.50-33205-80-6,172.16.0.1,33205,192.168.10.50,80,6,2017-07-07 17:15:21.598466,613,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4893.964111,3.065000e+02,3.500179e+02,554.0,59.0,613.0,6.130000e+02,0.000000e+00,613.0,613.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,3262.642741,1631.321370,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,613,Portscan,2017-07-07 17:15:21.599079
125262,172.16.0.1-192.168.10.50-35614-80-6,172.16.0.1,35614,192.168.10.50,80,6,2017-07-07 17:15:21.047424,2317,6,5,62.0,11576.0,62.0,0.0,10.333333,25.311394,5792.0,0.0,2315.2,2628.883261,5.022874e+06,4747.518343,2.317000e+02,2.824866e+02,865.0,4.0,2317.0,4.634000e+02,4.196085e+02,910.0,4.0,1189.0,2.972500e+02,4.257350e+02,931.0,46.0,1,1,0,0,1,0,200,168,2589.555460,2157.962883,0.0,5792.0,1058.000000,2052.698322,4.213570e+06,0,2,1,2,10,0,0,0,0.833333,1058.000000,10.333333,2315.2,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0

In [97]:
df_test['Label'].value_counts(dropna=False)

Label
Not Portscan    46527
Portscan        19087
Name: count, dtype: int64

In [92]:
flowmeter['Label'].value_counts()

Label
Not Portscan    387727
Portscan        159066
Name: count, dtype: int64

In [98]:
df_test = df_test.drop(columns='Timestamp End')

In [99]:
df_test.reset_index().to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated_test.csv", index=False)

In [100]:
df_train = flowmeter.drop(index=df_test.index).drop(columns='Timestamp End')
df_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448.0,1152.0,403.0,0.0,201.500000,204.724205,72.0,72.0,72.000000,0.000000,6.741133e+01,0.425756,2.398738e+06,5.798698e+06,16399772.0,3.0,112740690.0,3.636796e+06,6.848761e+06,16399772.0,3.0,112740348.0,7.516023e+06,8.323385e+06,16400110.0,3.0,16,16,0,0,0,0,1024,512,0.283837,0.141919,0.0,403.0,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.500000,158.333333,201.500000,72.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,164.571429,377,2078,16,16,32,0,3.594286e+02,1.199802e+01,380.0,343.0,1.610540e+07,4.988048e+05,16399772.0,15375229.0,-1,-1,16,8,24,112740690,Not Portscan
1,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448.0,5056.0,403.0,0.0,201.500000,204.724205,316.0,316.0,316.000000,0.000000,1.020396e+02,0.425756,2.398735e+06,5.798710e+06,16399782.0,2.0,112740560.0,3.636792e+06,6.848777e+06,16399782.0,2.0,112740242.0,7.516016e+06,8.323376e+06,16400110.0,4.0,16,16,0,0,0,0,1024,512,0.283838,0.141919,0.0,403.0,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.500000,239.666667,201.500000,316.000000,0.00,0.0,0.000000,0.0,0.0,0.0,4.571429,921.142857,2.285714,722.285714,955,2078,16,16,32,0,3.202857e+02,1.574499e+01,330.0,285.0,1.610543e+07,4.987937e+05,16399782.0,15375263.0,-1,-1,16,8,24,112740560,Not Portscan
2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,387,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,3.401977,2.947082e+05,1.772433e+06,23156061.0,0.0,113757377.0,2.947082e+05,1.772433e+06,23156061.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,0,0,3.401977,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,19.350000,0.000000,0.000000,0.000000,0,0,0,0,0,0,5.883193e+06,4.061994e+06,10999060.0,19.0,1.209364e+07,7.052789e+06,23156061.0,5505060.0,-1,-1,0,0,0,0,Not Portscan
3,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151.0,0.0,227.0,37.0,95.750000,55.785320,0.0,0.0,0.000000,0.000000,4.038274e+02,4.217519,2.377189e+05,1.511622e+06,19776791.0,0.0,91997219.0,2.377189e+05,1.511622e+06,19776791.0,0.0,0.0,0.000000e+00,0.000000e+

In [101]:
df_train.reset_index().to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated_train.csv", index=False)

In [102]:
df_test = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated_test.csv", parse_dates=['Timestamp'], date_format="mixed")
df_test

,index,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,93277,172.16.0.1-192.168.10.50-58377-80-6,172.16.0.1,58377,192.168.10.50,80,6,2017-07-07 17:15:45.540329,834,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,3597.122302,4.170000e+02,4.101219e+02,707.0,127.0,834.0,8.340000e+02,0.000000e+00,834.0,834.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2398.081535,1199.040767,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,834,Portscan
1,98282,172.16.0.1-192.168.10.50-62805-80-6,172.16.0.1,62805,192.168.10.50,80,6,2017-07-07 17:15:57.176074,674,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4451.038576,3.370000e+02,3.337544e+02,573.0,101.0,674.0,6.740000e+02,0.000000e+00,674.0,674.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2967.359050,1483.679525,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,674,Portscan
2,120211,172.16.0.1-192.168.10.50-33205-80-6,172.16.0.1,33205,192.168.10.50,80,6,2017-07-07 17:15:21.598466,613,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4893.964111,3.065000e+02,3.500179e+02,554.0,59.0,613.0,6.130000e+02,0.000000e+00,613.0,613.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,3262.642741,1631.321370,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,613,Portscan
3,125262,172.16.0.1-192.168.10.50-35614-80-6,172.16.0.1,35614,192.168.10.50,80,6,2017-07-07 17:15:21.047424,2317,6,5,62.0,11576.0,62.0,0.0,10.333333,25.311394,5792.0,0.0,2315.2,2628.883261,5.022874e+06,4747.518343,2.317000e+02,2.824866e+02,865.0,4.0,2317.0,4.634000e+02,4.196085e+02,910.0,4.0,1189.0,2.972500e+02,4.257350e+02,931.0,46.0,1,1,0,0,1,0,200,168,2589.555460,2157.962883,0.0,5792.0,1058.000000,2052.698322,4.213570e+06,0,2,1,2,10,0,0,0,0.833333,1058.000000,10.333333,2315.2,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,29200,28960,1,3,32,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,1,0,1,2317,Portscan
4,125265,172.16.0.1-192.168.10.50-35620-80-6,172.16.0.1,356

## Creating PCAP

In [103]:
flow_packets = []
with open(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_rerun_1_index.txt") as file:
    for line in file:
        indexes = line.strip().split(",")
        if len(indexes[-1]) == 0:
            indexes = indexes[:-1]
        indexes = [int(x) for x in indexes]
        flow_packets.append(indexes)
flow_packets

[[5,
  6,
  9,
  10,
  27,
  28,
  31,
  32,
  67,
  68,
  71,
  72,
  542,
  543,
  546,
  547,
  2842,
  2843,
  2846,
  2847,
  5024,
  5025,
  5028,
  5029,
  7653,
  7654,
  7657,
  7658,
  8803,
  8804,
  8807,
  8808,
  7,
  8,
  29,
  30,
  69,
  70,
  544,
  545,
  2844,
  2845,
  5026,
  5027,
  7655,
  7656,
  8805,
  8806],
 [11,
  12,
  15,
  16,
  33,
  34,
  37,
  38,
  73,
  74,
  77,
  78,
  548,
  549,
  552,
  553,
  2848,
  2849,
  2852,
  2853,
  5030,
  5031,
  5034,
  5035,
  7659,
  7660,
  7663,
  7664,
  8809,
  8810,
  8813,
  8814,
  13,
  14,
  35,
  36,
  75,
  76,
  550,
  551,
  2850,
  2851,
  5032,
  5033,
  7661,
  7662,
  8811,
  8812],
 [101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  191,
  192,
  193,
  194,
  195,
  196,
  19

In [104]:
df_test = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated_test.csv", parse_dates=['Timestamp'], date_format="mixed", index_col=0)
test_flow_indexes = list(df_test.index)

In [105]:
import itertools, gc

test_packet_idx = list(itertools.chain.from_iterable([flow_packets[i] for i in test_flow_indexes]))
test_packet_idx_set = set(test_packet_idx)

In [106]:
from scapy.all import PcapReader, wrpcap

with PcapReader(PCAP_INPUT) as pr:
    pcap_reader: PcapReader = pr

    out_packets = []

    cnt_pcap = 0
    i = 0
    while True:
        if i % 100_000 == 0:
            print(i)

        if len(out_packets) >= 500_000:
            print("SAVING...")
            wrpcap(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test_{cnt_pcap}.pcap", out_packets)
            del out_packets
            gc.collect()
            out_packets = []
            print(len(out_packets))
            cnt_pcap += 1

        try:
            packet = pcap_reader.read_packet()
        except:
            print(f"{i} packets have been iterated")
            break
        
        if i in test_packet_idx_set: # "not in" for training set
            out_packets.append(packet)

        i += 1

if len(out_packets) > 0:
    print("SAVING...")
    wrpcap(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test_{cnt_pcap}.pcap", out_packets)

print("DONE")

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
SAVING...
0
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
9997874 packets have been iterated
SAVING...
DONE


In [112]:
import os

# change to 0..N
command = f'mergecap -F pcap -w {OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test.pcap'
command += f' {OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test_0.pcap'
command += f' {OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test_1.pcap'

os.system(command)

0

In [113]:
rerun_test_flow_file, rerun_test_index_file = extract_flows_from_pcap(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_test.pcap", f"{BASE_OUTPUT_NAME}_rerun_test")
print(rerun_test_flow_file)
print(rerun_test_index_file)

[sudo] password for meervix: 

cic.cs.unb.ca.ifm.Cmd You select: /mnt/extra/datasets/processed/friday_test.pcap
cic.cs.unb.ca.ifm.Cmd Out folder: /mnt/extra/datasets/processed
cic.cs.unb.ca.ifm.Cmd CICFlowMeter received 1 pcap file
Working on... friday_test.pcap
friday_test.pcap is done. total 65600 flows 
Packet stats: Total=851797,Valid=851797,Discarded=0
-------------------------------------------------------------------------------
/mnt/extra/datasets/processed/friday_rerun_test_flow.csv
/mnt/extra/datasets/processed/friday_rerun_test_index.txt


In [114]:
rerun_test = pd.read_csv(rerun_test_flow_file, parse_dates=['Timestamp'], date_format="mixed")
rerun_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-205.174.165.73-7917-8080-6,192.168.10.9,7917,205.174.165.73,8080,6,2017-07-07 17:15:21.183058,561,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3565.062389,561.000000,0.000000e+00,561.0,561.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1782.531194,1782.531194,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,561,NeedManualLabel
1,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.401022,845,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2366.863905,845.000000,0.000000e+00,845.0,845.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1183.431953,1183.431953,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,845,NeedManualLabel
2,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.914385,777,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2574.002574,777.000000,0.000000e+00,777.0,777.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1287.001287,1287.001287,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,777,NeedManualLabel
3,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:37.780193,750,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2666.666667,750.000000,0.000000e+00,750.0,750.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1333.333333,1333.333333,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,750,NeedManualLabel
4,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:38.272414,478,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,4184.100418,478.000000,0.000000e+00,478.0,478.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,2092.050209,2092.050209,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,

In [115]:
rerun_test_keys = rerun_test[distinguishing_keys]
has_duplicate = (rerun_test_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

rerun_test = rerun_test[~rerun_test_keys.duplicated(keep=False)]
rerun_test

Has Duplicates? False


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-205.174.165.73-7917-8080-6,192.168.10.9,7917,205.174.165.73,8080,6,2017-07-07 17:15:21.183058,561,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3565.062389,561.000000,0.000000e+00,561.0,561.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1782.531194,1782.531194,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,561,NeedManualLabel
1,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.401022,845,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2366.863905,845.000000,0.000000e+00,845.0,845.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1183.431953,1183.431953,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,845,NeedManualLabel
2,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.914385,777,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2574.002574,777.000000,0.000000e+00,777.0,777.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1287.001287,1287.001287,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,777,NeedManualLabel
3,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:37.780193,750,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2666.666667,750.000000,0.000000e+00,750.0,750.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1333.333333,1333.333333,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,750,NeedManualLabel
4,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:38.272414,478,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,4184.100418,478.000000,0.000000e+00,478.0,478.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,2092.050209,2092.050209,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,

In [116]:
df_test_keys = df_test[distinguishing_keys + ['Label']]
has_duplicate = (df_test_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

df_test = df_test[~df_test_keys.duplicated(keep=False)]
df_test

Has Duplicates? False


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93277,172.16.0.1-192.168.10.50-58377-80-6,172.16.0.1,58377,192.168.10.50,80,6,2017-07-07 17:15:45.540329,834,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,3597.122302,4.170000e+02,4.101219e+02,707.0,127.0,834.0,8.340000e+02,0.000000e+00,834.0,834.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2398.081535,1199.040767,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,834,Portscan
98282,172.16.0.1-192.168.10.50-62805-80-6,172.16.0.1,62805,192.168.10.50,80,6,2017-07-07 17:15:57.176074,674,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4451.038576,3.370000e+02,3.337544e+02,573.0,101.0,674.0,6.740000e+02,0.000000e+00,674.0,674.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,2967.359050,1483.679525,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,674,Portscan
120211,172.16.0.1-192.168.10.50-33205-80-6,172.16.0.1,33205,192.168.10.50,80,6,2017-07-07 17:15:21.598466,613,2,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,4893.964111,3.065000e+02,3.500179e+02,554.0,59.0,613.0,6.130000e+02,0.000000e+00,613.0,613.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,1,0,44,24,3262.642741,1631.321370,0.0,0.0,0.000000,0.000000,0.000000e+00,0,2,1,0,1,0,0,0,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,1024,29200,0,0,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,613,Portscan
125262,172.16.0.1-192.168.10.50-35614-80-6,172.16.0.1,35614,192.168.10.50,80,6,2017-07-07 17:15:21.047424,2317,6,5,62.0,11576.0,62.0,0.0,10.333333,25.311394,5792.0,0.0,2315.2,2628.883261,5.022874e+06,4747.518343,2.317000e+02,2.824866e+02,865.0,4.0,2317.0,4.634000e+02,4.196085e+02,910.0,4.0,1189.0,2.972500e+02,4.257350e+02,931.0,46.0,1,1,0,0,1,0,200,168,2589.555460,2157.962883,0.0,5792.0,1058.000000,2052.698322,4.213570e+06,0,2,1,2,10,0,0,0,0.833333,1058.000000,10.333333,2315.2,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.00,0.00,0.0,29200,28960,1,3,32,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0

In [117]:
left_merge = rerun_test_keys.reset_index().merge(df_test_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.9,7917,205.174.165.73,8080,2017-07-07 17:15:21.183058,Not Portscan,both
1,192.168.10.15,55703,205.174.165.73,8080,2017-07-07 17:15:36.401022,Not Portscan,both
2,192.168.10.15,55703,205.174.165.73,8080,2017-07-07 17:15:36.914385,Not Portscan,both
3,192.168.10.8,6257,205.174.165.73,8080,2017-07-07 17:15:37.780193,Not Portscan,both
4,192.168.10.8,6257,205.174.165.73,8080,2017-07-07 17:15:38.272414,Not Portscan,both
...,...,...,...,...,...,...,...
65594,172.16.0.1,64318,192.168.10.50,2222,2017-07-07 18:09:11.499555,Portscan,both
65595,192.168.10.17,8398,192.168.10.3,53,2017-07-07 18:12:10.812610,Not Portscan,both
65596,172.16.0.1,41249,192.168.10.50,58080,2017-07-07 18:08:57.752078,Portscan,both


In [118]:
rerun_test_ignored = left_merge[left_merge['_merge'] == 'left_only']
rerun_test_ignored_idx = rerun_test_ignored.index
rerun_test = rerun_test.drop(index=rerun_test_ignored_idx)
rerun_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-205.174.165.73-7917-8080-6,192.168.10.9,7917,205.174.165.73,8080,6,2017-07-07 17:15:21.183058,561,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3565.062389,561.000000,0.000000e+00,561.0,561.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1782.531194,1782.531194,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,561,NeedManualLabel
1,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.401022,845,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2366.863905,845.000000,0.000000e+00,845.0,845.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1183.431953,1183.431953,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,845,NeedManualLabel
2,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.914385,777,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2574.002574,777.000000,0.000000e+00,777.0,777.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1287.001287,1287.001287,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,777,NeedManualLabel
3,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:37.780193,750,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2666.666667,750.000000,0.000000e+00,750.0,750.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1333.333333,1333.333333,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,750,NeedManualLabel
4,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:38.272414,478,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,4184.100418,478.000000,0.000000e+00,478.0,478.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,2092.050209,2092.050209,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,

In [119]:
rerun_test['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']
rerun_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-205.174.165.73-7917-8080-6,192.168.10.9,7917,205.174.165.73,8080,6,2017-07-07 17:15:21.183058,561,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3565.062389,561.000000,0.000000e+00,561.0,561.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1782.531194,1782.531194,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,561,Not Portscan
1,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.401022,845,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2366.863905,845.000000,0.000000e+00,845.0,845.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1183.431953,1183.431953,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,845,Not Portscan
2,192.168.10.15-205.174.165.73-55703-8080-6,192.168.10.15,55703,205.174.165.73,8080,6,2017-07-07 17:15:36.914385,777,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2574.002574,777.000000,0.000000e+00,777.0,777.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1287.001287,1287.001287,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,777,Not Portscan
3,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:37.780193,750,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,2666.666667,750.000000,0.000000e+00,750.0,750.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,1333.333333,1333.333333,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192,0,0,0,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,750,Not Portscan
4,192.168.10.8-205.174.165.73-6257-8080-6,192.168.10.8,6257,205.174.165.73,8080,6,2017-07-07 17:15:38.272414,478,1,1,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,4184.100418,478.000000,0.000000e+00,478.0,478.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,2092.050209,2092.050209,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.00,0.000000,0.000,0.0,0.

In [120]:
rerun_test['Label'].value_counts(dropna=False)

Label
Not Portscan    46483
Portscan        19073
Name: count, dtype: int64

In [121]:
rerun_test.reset_index().to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated_test_rerun.csv", index=False)